In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../EDA/dataframe_model/stretchedSociety_block_109_MAC005547.csv")

In [ ]:
df.head()

In [ ]:
df['time'] = pd.to_datetime(df['time'])

In [ ]:
df[['time','Energy_kwh','holiday','month','hour','dayofweek_num']].info()

In [ ]:
df = df[['time','Energy_kwh','holiday','month','hour','dayofweek_num']]

In [ ]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time

# Função para calcular MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

# Função para calcular SMAPE
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Preparar os dados
df_prophet = df.rename(columns={'time': 'ds', 'Energy_kwh': 'y'})

# Dividir os dados em treino e teste
train_size = len(df) - 168
df_train = df_prophet[:train_size]
df_test = df_prophet[train_size:]

# Criar e treinar o modelo Prophet
start_time = time.time()

model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
model.add_regressor('holiday')
model.add_regressor('month')
model.add_regressor('hour')
model.add_regressor('dayofweek_num')

model.fit(df_train)

training_time = time.time() - start_time

# Fazer previsões
future = df_test[['ds', 'holiday', 'month', 'hour', 'dayofweek_num']]
forecast = model.predict(future)

# Calcular métricas
y_true = df_test['y'].values
y_pred = forecast['yhat'].values

mae = mean_absolute_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)
smape = symmetric_mean_absolute_percentage_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

# Salvar resultados em CSV
results = pd.DataFrame({
    'model': ['Prophet'],
    'mae': [mae],
    'mape': [mape],
    'smape': [smape],
    'rmse': [rmse],
    'training_time': [training_time]
})

results.to_csv('prophet_results.csv', index=False)

print("Resultados salvos em 'prophet_results.csv'")
print(results)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time

# Função para calcular MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Função para calcular SMAPE
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Preparar os dados
df_sarimax = df.set_index('time')

# Dividir os dados em treino e teste
train_size = len(df) - 168
df_train = df_sarimax[:train_size]
df_test = df_sarimax[train_size:]

# Definir variáveis exógenas
exog_vars = ['holiday', 'month', 'hour', 'dayofweek_num']

# Criar e treinar o modelo SARIMAX
start_time = time.time()

# Aqui, estamos usando um modelo SARIMAX(1,1,1)(1,1,1,24) como exemplo
# Você pode ajustar esses parâmetros conforme necessário
model = SARIMAX(df_train['Energy_kwh'], 
                exog=df_train[exog_vars],
                order=(1, 1, 1),
                seasonal_order=(1, 1, 1, 24),
                enforce_stationarity=False,
                enforce_invertibility=False)

results = model.fit()

training_time = time.time() - start_time

# Fazer previsões
forecast = results.get_forecast(steps=168, exog=df_test[exog_vars])
y_pred = forecast.predicted_mean

# Calcular métricas
y_true = df_test['Energy_kwh'].values
mae = mean_absolute_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)
smape = symmetric_mean_absolute_percentage_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

# Salvar resultados em CSV
results_df = pd.DataFrame({
    'model': ['SARIMAX'],
    'mae': [mae],
    'mape': [mape],
    'smape': [smape],
    'rmse': [rmse],
    'training_time': [training_time]
})

results_df.to_csv('sarimax_results.csv', index=False)

print("Resultados salvos em 'sarimax_results.csv'")
print(results_df)
